# Entity Matching

In [29]:
import pandas as pd 
import textdistance as td 
import cleanco

# Preprocessing of Text

In [30]:
from cleanco import basename
business_name = "Some Big Pharma, LLC"
basename(business_name)
'Some Big Pharma'

'Some Big Pharma'

In [144]:
x = 'this is it'
len(x.split())

3

In [151]:
t = ['this is it','i am good']

In [153]:
[len(x.split()) for x in t]

[3, 3]

In [156]:
text_feats(['this is it'])

[3]


In [202]:
import re 

In [205]:
new_string = re.sub(r'[^\w\s]', '', a_string)

print(new_string)

apple inc


In [208]:
[re.sub(r'[^\w\s]', '', x) for x in ['tet','test', 'adf '] ]

['tet', 'test', 'adf ']

In [296]:
def lowerd_and_cleaned(l): 
    lowered = [x.lower() for x in l]
    cleaned = [re.sub(r'[^\w\s]', '', x) for x in lowered]
    return cleaned

def text_feats(l):
    lens = [len(x) for x in l]
    word_len = [len(x.split()) for x in l]
    
    len_diff = abs(lens[0] - lens[1])
    words_diff = abs(word_len[0] - word_len[1])
    return len_diff, words_diff
    
def base_name(l): 
    return [cleanco.basename(x) for x in l]

def ultra_base(l): 
    return [cleanco.basename(cleanco.basename(x)) for x in l]
    
def is_exact(x,y): 
    return x==y

def distance(l): 
    jaro = td.jaro_winkler(l[0], l[1])
    jaccard = td.jaccard(l[0], l[1])
    return jaro, jaccard

def fit(x,y): 
    
    lower = lowerd_and_cleaned([x,y])
    base = base_name(lower)
    u_base = ultra_base(lower)
    
    
    
    jaro, jaccard = distance(lower)
    jaro_base, jaccard_base = distance(base)
    jaro_u_base, jaccard_u_base = distance(u_base)
    
    exact = is_exact(lower[0],lower[1])
    base_exact = is_exact(base[0], base[1])
    len_diff, word_diff = text_feats(lower)
    # print(x, y)
    # print('\n\n')
    # print(lower)
    return pd.DataFrame(
    {
        'Jaro':jaro, 
        'Jaccard':jaccard,
        'Jaro Base':jaro_base,
        'Jaccard Base':jaccard_base,
        'Jaro U Base':jaro_u_base,
        'Jaccard U Base':jaccard_u_base,
        'Exact':int(exact),
        'Base Exact':int(base_exact),
        'Diff Str Len':len_diff,
        'Diff Word Len':word_diff
    }, index = range(1))

In [297]:
print(f'{td.jaro_winkler("matt", "johnmatt")}')
print(f'{td.jaccard("matt","johnmatt")}')

0.4583333333333333
0.5


In [298]:
fit('Apple, corp', 'apple, inc')

,Jaro,Jaccard,Jaro Base,Jaccard Base,Jaro U Base,Jaccard U Base,Exact,Base Exact,Diff Str Len,Diff Word Len
0,0.895556,0.583333,1,1,1,1,0,1,1,0


# Test data

In [299]:
df = pd.read_csv('room_type.csv')

In [301]:
from tqdm import tqdm

In [302]:
x = pd.DataFrame()
for i in tqdm(range(len(df))): 
    d = fit(df['Expedia'][i], df['Booking.com'][i])
    x = pd.concat([x, d])

100%|███████████████████████████████████████| 103/103 [00:00<00:00, 1171.68it/s]


In [303]:
x

,Jaro,Jaccard,Jaro Base,Jaccard Base,Jaro U Base,Jaccard U Base,Exact,Base Exact,Diff Str Len,Diff Word Len
0,0.895455,0.727273,0.895455,0.727273,0.895455,0.727273,0,0,6,2
0,0.869323,0.697674,0.869323,0.697674,0.869323,0.697674,0,0,3,1
0,0.933333,0.666667,0.933333,0.666667,0.933333,0.666667,0,0,11,3
0,0.492091,0.739130,0.492091,0.739130,0.492091,0.739130,0,0,6,2
0,0.934007,0.818182,0.934007,0.818182,0.934007,0.818182,0,0,6,2
...,...,...,...,...,...,...,...,...,...,...
0,0.740019,0.777778,0.752698,0.790323,0.752698,0.790323,0,0,10,1
0,0.747646,0.696970,0.757025,0.730159,0.757025,0.730159,0,0,18,2
0,0.627901,0.736842,0.627901,0.736842,0.627901,0.736842,0,0,4,0
0,0.733872,0.738095,0.733872,0.738095,0.733872,0.738095,0,0,5,1


In [304]:
df

,Expedia,Booking.com
0,"Deluxe Room, 1 King Bed",Deluxe King Room
1,"Standard Room, 1 King Bed, Accessible",Standard King Roll-in Shower Accessible
2,"Grand Corner King Room, 1 King Bed",Grand Corner King Room
3,"Suite, 1 King Bed (Parlor)",King Parlor Suite
4,"High-Floor Premium Room, 1 King Bed",High-Floor Premium King Room
...,...,...
98,"Room, 1 King Bed, Accessible, Resort View (Ali...",Alii Tower Resort View With King Bed - Mobilit...
99,"Room, 1 King Bed, Accessible, View (Rainbow, B...",Rainbow Tower Ocean View With King Bed - Mobil...
100,"Room, 1 King Bed, Ocean View (Alii)",Alii Tower Ocean View With King Bed
101,"Room, 1 King Bed, Oceanfront (Rainbow)",Rainbow Tower Ocean Front with King Bed
